<a href="https://colab.research.google.com/github/Edisuism/Machine_Learning/blob/master/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2


In this page, I will develop a machine learning algorithm from scratch and then test it on a sample dataset

Steps involved:
- Find sample dataset
- Choose learner algorithm and note down input/output
- Preprocess sample dataset to input requirements and split into test/training sets 
- code learner algorithm **(only this needs to be coded, preprocessing can use from libaries like SKlearn)**
- train algorithm and then apply to test set

# Introduction

The algorithm I have decided to build from scratch is an ID3 Decision tree. This algorithm takes in variables as well in addition to their attributes such as a flower and it's colour and size. As output, the algorithm will determine based on the attributes given and the target variable it has been tasked with estimating, what the target variable is. In the case of the flower it could determine what type of flower it is based on it's attributes. This works by first determining the most impactful attribute in deciding the variable type as the root node and then repeating the process for all the other attributes, achieving a greater level of classification each time.


# Exploration

Some Challenges I find in creating this algorithm is figuring out how to turn the theory into practical, functional code such as calculating entropy with respects to each attribute and it's result on the target variable while ensuring it repeats the process for each layer of the decision tree.


My current plan for the project is to split the dataframe into a training and test set to ensure I will be able to test the algorithm without any additional data. 


As for the algorithm itself, I will calculate the entropy of the target variable of the data set and then for every attribute with the addition of also calculating the information gain of the attributes by subtracting the entropy from the entropy of the target variable. After calculating the information gain for each attribute, the algorithm will choose the highest value as the root node and then cull all pure results from the training dataframe, that is, if all the results are to play tennis if the outlook is overcast, all those rows will be removed from the table. The algorithm will then recalculate the new entropy of the target variables and each attribute as it did to find the root node. This process of calculating entropy of the target variable and the information gain of attributes will repeat until the data is pure.


However, what happens in the case that the tree has already reached the end of it's possibilities (taking note that the tree will not go up a level again in order to test out new possibilities)? I am currently thinking of simply semi-randomising it's estimation of the target variable based on the quantity of the results e.g. if at the end of the tree, there is still 1 result saying to play tennis and 1 result saying to not play tennis, then the answer will have a 50% chance to be either play or not play.

# Methodology

Import data


In [59]:
import numpy as np
import pandas as pd
from numpy import log2 as log
import random
import pprint

url = 'https://raw.githubusercontent.com/Edisuism/Machine_Learning/master/play_tennis.csv'
df = pd.read_csv(url)
print (len(df))
df.head()

14


,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes


Split into training and testing sets

In [0]:
def train_test_split(df, test_size):
  test_size = round(test_size * len(df)) 
  indices = df.index.tolist()
  
  #randomly select samples based on test_size
  test_indices = random.sample(population = indices, k = test_size)
  
  #create test and training set
  test_df = df.loc[test_indices]
  train_df = df.drop(test_indices)
  
  return train_df, test_df



In [0]:
train_df, test_df = train_test_split(df, test_size = 0.3)

In [0]:
print (len(train_df))
train_df.head()

In [0]:
print (len(test_df))
test_df.head()

Find Entropy of Class


In [62]:
#calculate the entropy of the class
def class_entropy(df):
  
#target is the column we are interested in  
  target = df.keys()[-1]
  entropy_class = 0  
  
#get possible values for the target, yes and no
  values = df[target].unique()  
  for value in values:
      fraction = df[target].value_counts()[value]/len(df[target])  
      entropy_class += -fraction*np.log2(fraction)
  return entropy_class

class_entropy(train_df)

1.0

Find Entropy and Information Gain of Attributes

In [63]:
#calculate entropy of each attribute
def attribute_entropy(df,target_attribute):
  target = df.keys()[-1]
  attribute = target_attribute
#in case of a 0 denominator to prevent error
  eps = np.finfo(float).eps 
  
#get unique values of the chosen attribute and calculate entropy based on the variance in target
  target_variables = df[target].unique() 
  variables = df[attribute].unique()    
  entropy_attribute = 0
  for variable in variables:
      entropy_each_feature = 0
      for target_variable in target_variables:
          num = len(df[attribute][df[attribute]==variable][df[target] ==target_variable]) 
          den = len(df[attribute][df[attribute]==variable])  
          fraction = num/(den+eps)  
          
#entropy for one feature 
          entropy_each_feature += -fraction*log(fraction+eps) 
      fraction2 = den/len(df)
      
#all the entropy for attribute
      entropy_attribute += -fraction2*entropy_each_feature  
  E_final = abs(entropy_attribute)
  return E_final

print (attribute_entropy(train_df, 'outlook'))
print (attribute_entropy(train_df, 'temp'))
print (attribute_entropy(train_df, 'humidity'))
print (attribute_entropy(train_df, 'wind'))

0.3245112497836527
0.9999999999999993
0.8754887502163462
0.9999999999999993


Find Highest Info Gain Attribute

In [64]:
#calculate and compare information gain of each attribute and returns the one with the highest value
def calculate_winner(df):
  IG = []
  
#1 to class because [0] is just an index
  for key in df.keys()[1:-1]: 
    IG.append(class_entropy(df)-attribute_entropy(df,key))
  
  return df.keys()[1:-1][np.argmax(IG)]

calculate_winner(train_df)

'outlook'

Splitting function

In [0]:
#function that can be repeatedly called to split the table
def split_table(df, node, value):
  return df[df[node] == value].reset_index(drop=True)

Build Tree

In [0]:
def buildTree(df,tree=None): 
    target = df.keys()[-1] 
    
#calculate root node based on current table
    node = calculate_winner(df)
    attValue = np.unique(df[node])
    
#create tree
    if tree is None:                    
        tree={}
        tree[node] = {}
        
#create a new subtable for each attribute value
    for value in attValue:
        subtable = split_table(df, node, value)
        clValue, counts = np.unique(subtable[target],return_counts=True)                        
        
#when data is pure then stop otherwise continue to build the tree    
        if len(counts)==1:
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable)
                   
    return tree
  
  

In [51]:
decision_tree = buildTree(train_df)
pprint.pprint(decision_tree)

{'humidity': {'High': {'outlook': {'Overcast': 'Yes',
                                   'Rain': {'wind': {'Strong': 'No',
                                                     'Weak': 'Yes'}},
                                   'Sunny': 'No'}},
              'Normal': 'Yes'}}


Prediction

In [0]:
def predict(inst,tree):
    for nodes in tree.keys():        
        value = inst[nodes]
        tree = tree[nodes][value]
        
#default prediction
        prediction = 0
        
#find corresponding values in decision tree
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;                            
        
    return prediction

In [97]:
def predict_test(df, tree):
#create new list to store predictions
  prediction_list = []
  i=0
  
#for each row, get the prediction from the tree based on attributes and add to list
  for i in range (len(df.index)):
    row = df.iloc[i]
    prediction = predict(row,tree)
    prediction_list.append(prediction)
  
  return prediction_list

predict_test(test_df, decision_tree)

['Yes', 'Yes', 'Yes', 'Yes']

In [123]:
def accuracy(df, predicted):
  target = df.keys()[-1]
  
#get the values of the target variable in list form
  actual = test_df[target].tolist()
  correct = 0
  
#compare actual values of dataframe to predicted values of dataframe
  for i in range (len(df.index)):
    if actual[i] == predicted[i]:
      correct += 1      
      
#calculate the percentage of predictions that were correct 
  return correct / float(len(actual)) * 100.0

accuracy(test_df, predict_test(test_df, decision_tree))

100.0

Notes:
Gini index = how much uncertainty there is in a node
,Information gain = how much uncertainty is removed in a node

# Evaluation

# Conclusion

# Ethical

My algorithm in it's simplest form is taking in statistical information as input and then creating a prediction based on them as output. This can have many applications such as in determining a person's likelihood to commit crime and also determine the driving patterns of self-driving vehicles. In these scenarios, many ethical misuses of the algorithm could be made such as giving greater weight to certain attributes as well as the manipulation of input data to create biased decision trees in order to support a certain purpose. As an example, a person with brown hair who plans to steal may reduce the weighting of brown hair in determining one's propensity to commit crime or sample from biased data which results in brown hair having no or less impact on one's propensity to commit crime. By following the ethical framework of Utilitarianism where communal happiness forms moral decisions, we are able to deem this misuse unethical. This is because by altering the decision making process of the machine to favor brown haired people, more statistically likely people to commit crime are getting away with criminal activity resulting in a less stable and ultimately less happy society. This method of thinking however is not perfect as 'level of happiness' a hard thing to measure especially with the action of tampering with a machine learning algorithm as the results of doing so are not obviously clear. It is then important to patch up such grey areas with the Contractarianism framework which declares morality as that which a group determines to be so. By first valueing universal net happiness first and then falling back onto hard contracts next, morals become very clear. Applying this to aforementioned 'brown hair' example... by changing the algorithm in favor of brown haired people, though it is hard to determine whether it creates the maximum net happiness, the person tampering with the algorithm is likely in breach of company contracts that prevent personal intervention and thus his action is deemed unethical.